In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
uploaded_files = [f"{year}-{str(year + 1)[-2:]}.csv" for year in range(2019, 1999, -1)]
uploaded_files.reverse()
uploaded_files.append('2020-2021.csv')
uploaded_files.append('2021-2022.csv')

In [3]:
renamed_dataframes = {}

for file in uploaded_files:
    
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.rename(columns={'Referee': 'Ref'})
    df['Game_settled'] = (df['FTR'] != 'D').astype(int)
    
    season = file.split('.')[0].replace('-', '')  
    
    unique_name = f"s_{season}"  
    
    df['Season'] = season
    
    renamed_dataframes[unique_name] = df

    print(f"File '{file}' has been read and renamed to '{unique_name}'")
    
    list_of_frame_names = []
    
for name, dataframe in renamed_dataframes.items():
    print(f"DataFrame name: {name}")
    list_of_frame_names.append(name)
    
for key, value in renamed_dataframes.items():
    globals()[key] = value

File '2000-01.csv' has been read and renamed to 's_200001'
File '2001-02.csv' has been read and renamed to 's_200102'
File '2002-03.csv' has been read and renamed to 's_200203'
File '2003-04.csv' has been read and renamed to 's_200304'
File '2004-05.csv' has been read and renamed to 's_200405'
File '2005-06.csv' has been read and renamed to 's_200506'
File '2006-07.csv' has been read and renamed to 's_200607'
File '2007-08.csv' has been read and renamed to 's_200708'
File '2008-09.csv' has been read and renamed to 's_200809'
File '2009-10.csv' has been read and renamed to 's_200910'
File '2010-11.csv' has been read and renamed to 's_201011'
File '2011-12.csv' has been read and renamed to 's_201112'
File '2012-13.csv' has been read and renamed to 's_201213'
File '2013-14.csv' has been read and renamed to 's_201314'
File '2014-15.csv' has been read and renamed to 's_201415'
File '2015-16.csv' has been read and renamed to 's_201516'
File '2016-17.csv' has been read and renamed to 's_20161

In [4]:
seasons_to_drop = ['s_201819', 's_201920']
renamed_dataframes = {key: value for key, value in renamed_dataframes.items() if key not in seasons_to_drop}

In [5]:
s_20202021['Date'] = s_20212022['Date'].fillna(method='ffill')

In [6]:
i = 1
columns_to_keep = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'Game_settled', 'FTAG', 'FTR', 'B365H', 'B365D', 'B365A', 'Season']

for key in renamed_dataframes.keys():
    print(i)
    print(key)
    i += 1
    
    frame = renamed_dataframes[key]
    
    existing_columns = [col for col in columns_to_keep if col in frame.columns]
    frame = frame.loc[:, existing_columns]
    
    renamed_dataframes[key] = frame
    
    print(frame.columns)  
    print(frame.columns[frame.isna().any()])  
    print(len(frame))  
    
for key, value in renamed_dataframes.items():
    globals()[key] = value

1
s_200001
Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'Game_settled', 'FTAG', 'FTR',
       'Season'],
      dtype='object')
Index([], dtype='object')
380
2
s_200102
Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'Game_settled', 'FTAG', 'FTR',
       'Season'],
      dtype='object')
Index([], dtype='object')
380
3
s_200203
Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'Game_settled', 'FTAG', 'FTR',
       'Season'],
      dtype='object')
Index([], dtype='object')
380
4
s_200304
Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'Game_settled', 'FTAG', 'FTR',
       'Season'],
      dtype='object')
Index([], dtype='object')
380
5
s_200405
Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'Game_settled', 'FTAG', 'FTR',
       'Season'],
      dtype='object')
Index([], dtype='object')
380
6
s_200506
Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'Game_settled', 'FTAG', 'FTR',
       'Season'],
      dtype='object')
Index([], dtype='object')
380
7
s_200607
Index(['Date', 'HomeTeam', 'AwayTea

In [7]:
def calc_bet_odds_diff_std(df):
    """
    Calculates the difference between betting odds for home and away teams,
    then categorizes the difference into bins with corresponding labels,
    and standardizes the differences.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing 'B365H' and 'B365A' columns.

    Returns:
    - pd.DataFrame: The updated DataFrame with 'Bet_diff', 'Bet_diff_bin', 
      and 'Bet_diff_standardized' columns added.
    """
    df['Bet_diff'] = df['B365H'] - df['B365A']
    
    bins = [-50, -18, -13, -10, -7.5, -5, -3, -2, -1, 0, 1, 2, 3, 5, 7.5, 10, 13, 18, 50]
    labels = [-9, -8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    
    df['Bet_diff_bin'] = pd.cut(df['Bet_diff'], bins=bins, labels=labels)
    
    mean_diff = df['Bet_diff'].mean()
    std_diff = df['Bet_diff'].std()
    df['Bet_diff_std'] = (df['Bet_diff'] - mean_diff) / std_diff
    
    return df

In [8]:
def calc_cur_pos_diff_std(df):
    """
    Determines the current points and league standings for each team after every matchday, calculates the positional differences,
    and adds a standardized column for these differences.

    Parameters:
    - df (pd.DataFrame): Input DataFrame with 'HomeTeam', 'AwayTeam', and 'FTR' columns.

    Returns:
    - pd.DataFrame: Updated DataFrame with additional columns including 'Pos_diff_std'.
    """
    team_points = {team: 0 for team in pd.unique(df[['HomeTeam', 'AwayTeam']].values.ravel())}
    home_curpts = []
    away_curpts = []

    for _, row in df.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        result = row['FTR']

        home_curpts.append(team_points[home_team])
        away_curpts.append(team_points[away_team])

        if result == 'H':
            team_points[home_team] += 3
        elif result == 'A':
            team_points[away_team] += 3
        elif result == 'D':
            team_points[home_team] += 1
            team_points[away_team] += 1

    df['HomeTeam_curpts'] = home_curpts
    df['AwayTeam_curpts'] = away_curpts

    home_curpos = []
    away_curpos = []
    all_teams = pd.unique(df[['HomeTeam', 'AwayTeam']].values.ravel())

    for index, row in df.iterrows():
        
        current_points = {team: team_points.get(team, 0) for team in all_teams}
        rankings = sorted(current_points.items(), key=lambda x: (-x[1], x[0]))
        team_positions = {team: rank + 1 for rank, (team, _) in enumerate(rankings)}

        home_curpos.append(team_positions[row['HomeTeam']])
        away_curpos.append(team_positions[row['AwayTeam']])

    df['HomeTeam_curpos'] = home_curpos
    df['AwayTeam_curpos'] = away_curpos

    df['Pos_diff'] = df['HomeTeam_curpos'] - df['AwayTeam_curpos']
    df['Pos_diff_abs'] = df['Pos_diff'].abs()

    bins = [-21, -15.5, -12.5, -9.5, -6.5, -3.5, -1.5, 1.5, 3.5, 6.5, 9.5, 12.5, 15.5, 21]
    labels = [-6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6]
    df['Pos_diff_bin'] = pd.cut(df['Pos_diff'], bins=bins, labels=labels)

    mean_diff = df['Pos_diff'].mean()
    std_diff = df['Pos_diff'].std()
    df['Pos_diff_std'] = (df['Pos_diff'] - mean_diff) / std_diff

    return df

In [9]:
def calc_cur_pts_avg_diff_std(df):
    """
    Monitors and computes the running average points for each team after each matchday, calculates the differences between these averages,
    and adds a standardized column for these differences.

    Parameters:
    - df (pd.DataFrame): The input DataFrame with 'HomeTeam', 'AwayTeam', and 'FTR' columns.

    Returns:
    - pd.DataFrame: The updated DataFrame with additional columns including 'Avg_pts_diff_std'.
    """
    team_points = {team_id: 0 for team_id in pd.unique(df[['HomeTeam', 'AwayTeam']].values.ravel())}
    team_games = {team_id: 0 for team_id in pd.unique(df[['HomeTeam', 'AwayTeam']].values.ravel())}

    home_curpts_avg = []
    away_curpts_avg = []

    for index, row in df.iterrows():
        home_id = row['HomeTeam']
        away_id = row['AwayTeam']
        result = row['FTR']
        
        home_avg = round(team_points[home_id] / team_games[home_id], 2) if team_games[home_id] > 0 else 0
        away_avg = round(team_points[away_id] / team_games[away_id], 2) if team_games[away_id] > 0 else 0
        
        home_curpts_avg.append(home_avg)
        away_curpts_avg.append(away_avg)
        
        if result == 'H':  # Home Win
            team_points[home_id] += 3
        elif result == 'A':  # Away Win
            team_points[away_id] += 3
        elif result == 'D':  # Draw
            team_points[home_id] += 1
            team_points[away_id] += 1

        team_games[home_id] += 1
        team_games[away_id] += 1

    df['HomeTeam_curpts_avg'] = home_curpts_avg
    df['AwayTeam_curpts_avg'] = away_curpts_avg
    df['Avg_pts_diff'] = df['HomeTeam_curpts_avg'] - df['AwayTeam_curpts_avg']
    
    bins = [-3.5, -2.5, -1.5, -0.5, 0.5, 1.5, 2.5, 3.5]
    labels = [-3, -2, -1, 0, 1, 2, 3]
    df['Avg_pts_diff_bin'] = pd.cut(df['Avg_pts_diff'], labels=labels, bins=bins)
    
    mean_diff = df['Avg_pts_diff'].mean()
    std_diff = df['Avg_pts_diff'].std()
    df['Avg_pts_diff_std'] = (df['Avg_pts_diff'] - mean_diff) / std_diff

    return df

In [10]:
def calc_avg_goals_scored_diff_std(df):
    """
    Calculates the average goals scored by each team, updating the values after each matchday. Then computes the difference 
    between the averages for the home and away teams, assigns labels based on the results, and standardizes the differences.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing columns 'HomeTeam', 'AwayTeam', 'FTHG', and 'FTAG'.

    Returns:
    - pd.DataFrame: Updated DataFrame with additional columns.
    """

    all_teams = pd.unique(df[['HomeTeam', 'AwayTeam']].values.ravel())

    team_goals_avg = {team: 0 for team in all_teams}  # Cumulative goals scored
    team_games_played = {team: 0 for team in all_teams}  # Number of games played

    home_team_avg_scored = []
    away_team_avg_scored = []

    for index, row in df.iterrows():

        home_team = row['HomeTeam']
        away_team = row['AwayTeam']

        team_goals_avg[home_team] += row['FTHG']
        team_games_played[home_team] += 1
        home_avg = round(team_goals_avg[home_team] / team_games_played[home_team], 2)
        home_team_avg_scored.append(home_avg)

        team_goals_avg[away_team] += row['FTAG']
        team_games_played[away_team] += 1
        away_avg = round(team_goals_avg[away_team] / team_games_played[away_team], 2)
        away_team_avg_scored.append(away_avg)

    df['HomeTeam_avg_scored'] = home_team_avg_scored
    df['AwayTeam_avg_scored'] = away_team_avg_scored
    df['Avg_goals_scored_diff'] = df['HomeTeam_avg_scored'] - df['AwayTeam_avg_scored']

    mean_diff = df['Avg_goals_scored_diff'].mean()
    std_diff = df['Avg_goals_scored_diff'].std()
    df['Avg_goals_scored_diff_std'] = (df['Avg_goals_scored_diff'] - mean_diff) / std_diff

    bins = [-10, -2.8, -2.1, -1.1, -0.6, -0.2, 0.2, 0.6, 1.1, 2.1, 2.8, 10]
    labels = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
    df['Avg_goals_scored_diff_bin'] = pd.cut(df['Avg_goals_scored_diff'], labels=labels, bins=bins)

    return df

In [11]:
def calc_draw_ratio_diff_std(df):
    """
    Calculates the 'draws-to-games-played' ratio for each team, updating the values after each matchday.
    Then computes the difference between these ratios and adds a standardized version of the difference.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing columns 'HomeTeam', 'AwayTeam', and 'FTR'.

    Returns:
    - pd.DataFrame: Updated DataFrame with additional columns.
    """


    all_teams = pd.unique(df[['HomeTeam', 'AwayTeam']].values.ravel())


    team_draws = {team: 0 for team in all_teams}  
    team_games_played = {team: 0 for team in all_teams}  

    home_team_ratio = []
    away_team_ratio = []

    for _, row in df.iterrows():
        
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']

        team_games_played[home_team] += 1
        team_games_played[away_team] += 1

        if row['FTR'] == 'D':
            team_draws[home_team] += 1
            team_draws[away_team] += 1
            
        home_ratio = (
            team_draws[home_team] / team_games_played[home_team]
            if team_games_played[home_team] > 0
            else 0
        )
        away_ratio = (
            team_draws[away_team] / team_games_played[away_team]
            if team_games_played[away_team] > 0
            else 0
        )

        home_team_ratio.append(round(home_ratio, 2))
        away_team_ratio.append(round(away_ratio, 2))

    df['HomeTeam_ratio'] = home_team_ratio
    df['AwayTeam_ratio'] = away_team_ratio

    df['Draw_ratio_diff'] = df['HomeTeam_ratio'] - df['AwayTeam_ratio']

    mean_diff = df['Draw_ratio_diff'].mean()
    std_diff = df['Draw_ratio_diff'].std()
    df['Draw_ratio_diff_std'] = (df['Draw_ratio_diff'] - mean_diff) / std_diff

    return df

In [12]:
def calc_team_form_diff_std(df):
    """
    Calculates the rolling form (total points over the last 3 games) for each team, adding the results as 'HomeForm' and 'AwayForm' columns to the DataFrame. Then determines the difference between these values and categorizes the results into predefined bins.

    Parameters:
    - df (pd.DataFrame): Input DataFrame containing columns 'HomeTeam', 'AwayTeam', and 'FTR'.
        
    Returns:
    - pd.DataFrame: Updated DataFrame with added 'HomeForm', 'AwayForm', 'Form_diff',  and 'Form_diff_bin' columns.
    """
    df['HomePoints'] = df['FTR'].apply(lambda x: 3 if x == 'H' else 1 if x == 'D' else 0)
    df['AwayPoints'] = df['FTR'].apply(lambda x: 3 if x == 'A' else 1 if x == 'D' else 0)

    team_form = pd.concat([df[['HomeTeam', 'HomePoints']].rename(columns={'HomeTeam': 'Team', 'HomePoints': 'Points'}),
            df[['AwayTeam', 'AwayPoints']].rename(columns={'AwayTeam': 'Team', 'AwayPoints': 'Points'})], axis=0).sort_index(kind="merge").reset_index(drop=True)

    team_form['OriginalIndex'] = team_form.index

    team_form['Form'] = team_form.groupby('Team')['Points'].transform(
            lambda x: x.shift(1).rolling(window=3, min_periods=1).sum()
        )
    team_form = team_form.sort_values('OriginalIndex')
    team_form = team_form.drop(columns = ['Points', 'OriginalIndex'])

    reshaped_data = {
            'HomeTeam': team_form.iloc[::2, 0].values,
            'AwayTeam': team_form.iloc[1::2, 0].values,
            'HomeForm': team_form.iloc[::2, 1].values,
            'AwayForm': team_form.iloc[1::2, 1].values,
        }

    reshaped_team_form = pd.DataFrame(reshaped_data)

    merged_df = pd.concat([df, reshaped_team_form.drop(columns=['HomeTeam', 'AwayTeam'])], axis=1)
    
    bins = [-9.5, -7.5, -5.5, -3.5, -1.5, 1.5, 3.5, 5.5, 7.5, 9.5]
    labels = [-4, -3, -2, -1, 0, 1, 2, 3, 4]
    merged_df['Form_diff'] = merged_df['HomeForm'] - merged_df['AwayForm']
    merged_df['Form_diff_bin'] = pd.cut(merged_df['Form_diff'], labels=labels, bins=bins)

    mean_diff = merged_df['Form_diff'].mean()
    std_diff = merged_df['Form_diff'].std()
    merged_df['Form_diff_std'] = (merged_df['Form_diff'] - mean_diff) / std_diff

    return merged_df    

In [13]:
list_of_methods_game_set = [
    calc_cur_pos_diff_std, 
    calc_cur_pts_avg_diff_std, 
    calc_avg_goals_scored_diff_std, 
    calc_draw_ratio_diff_std,
    calc_team_form_diff_std
]

In [14]:
renam_df_copy = dict(renamed_dataframes.copy())

Creating table for 'is game settled' prediction

In [15]:
for key, value in renam_df_copy.items():
    
    for method in list_of_methods_game_set:

        renam_df_copy[key] = method(value) 
    
    renam_df_copy[key] = renam_df_copy[key].filter(regex='_std|^Game_settled$')
   
    renam_df_copy[key] = renam_df_copy[key].iloc[30:]
    
    print(f"DataFrame: {key}")
    print(renam_df_copy[key].columns)   
    print(renam_df_copy[key].shape)
    
final_frame_for_game_set = pd.concat(renam_df_copy.values(), ignore_index=True)

DataFrame: s_200001
Index(['Game_settled', 'Pos_diff_std', 'Avg_pts_diff_std',
       'Avg_goals_scored_diff_std', 'Draw_ratio_diff_std', 'Form_diff_std'],
      dtype='object')
(350, 6)
DataFrame: s_200102
Index(['Game_settled', 'Pos_diff_std', 'Avg_pts_diff_std',
       'Avg_goals_scored_diff_std', 'Draw_ratio_diff_std', 'Form_diff_std'],
      dtype='object')
(350, 6)
DataFrame: s_200203
Index(['Game_settled', 'Pos_diff_std', 'Avg_pts_diff_std',
       'Avg_goals_scored_diff_std', 'Draw_ratio_diff_std', 'Form_diff_std'],
      dtype='object')
(350, 6)
DataFrame: s_200304
Index(['Game_settled', 'Pos_diff_std', 'Avg_pts_diff_std',
       'Avg_goals_scored_diff_std', 'Draw_ratio_diff_std', 'Form_diff_std'],
      dtype='object')
(350, 6)
DataFrame: s_200405
Index(['Game_settled', 'Pos_diff_std', 'Avg_pts_diff_std',
       'Avg_goals_scored_diff_std', 'Draw_ratio_diff_std', 'Form_diff_std'],
      dtype='object')
(350, 6)
DataFrame: s_200506
Index(['Game_settled', 'Pos_diff_std', 'Avg_p

In [16]:
final_frame_for_game_set

Game_settled  Pos_diff_std  ...  Draw_ratio_diff_std  Form_diff_std
0                1     -2.267944  ...             1.683673       0.415914
1                0     -1.193655  ...            -1.674835       0.728695
2                1     -0.358096  ...             0.004419       0.103133
3                0     -0.119365  ...            -3.354089       0.415914
4                0      0.477462  ...             1.683673      -0.522429
...            ...           ...  ...                  ...            ...
6995             1      0.716193  ...             0.698780       0.042142
6996             1     -0.835558  ...            -0.410042       1.991221
6997             1     -0.954924  ...             0.276372       1.434342
6998             1     -1.551751  ...             0.012367       1.434342
6999             1      1.909848  ...             0.276372      -1.628497

[7000 rows x 6 columns]

In [17]:
data = final_frame_for_game_set

features = ['Pos_diff_std', 'Avg_pts_diff_std', 'Avg_goals_scored_diff_std', 'Form_diff_std', 'Draw_ratio_diff_std']
X = data[features]  
y = data['Game_settled']   

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7221428571428572
              precision    recall  f1-score   support

           0       0.24      0.04      0.07       356
           1       0.74      0.95      0.84      1044

    accuracy                           0.72      1400
   macro avg       0.49      0.50      0.45      1400
weighted avg       0.62      0.72      0.64      1400

